In [6]:
import re
import requests
from urllib.parse import urlsplit
from collections import deque
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import files
import xlrd


import pandas as pd
cols_list = ["URLS"]
links = pd.read_csv("links.csv", usecols=cols_list) 
links = links["URLS"]

duplicate = deque()
for i in range(len(links)):
  link = links[i]
  duplicate.append(link)
#duplicate


scraped = set()
emails = set()
final_emails = []
for t in range(len(duplicate)):
  original_url = duplicate[t]
  unscraped = deque([original_url])
  emails.clear()
  count =0
  while len(unscraped) :
      
      if(count>10):
        break
      
      url = unscraped.popleft()  
      # print("unscraped is" )
      # print(unscraped)
      scraped.add(url)

      parts = urlsplit(url)
          
      base_url = "{0.scheme}://{0.netloc}".format(parts)
      if '/' in parts.path:
        path = url[:url.rfind('/')+1]
      else:
        path = url

      print("Crawling URL %s" % url)
      try:
          response = requests.get(url)
      except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
          continue

      soup = BeautifulSoup(response.text, 'lxml')
      
      for anchor in soup.find_all("a"):
        if anchor.text != "Contact Us" and anchor.text != "About Us":
          continue

        #print(anchor.text)

        
        
        if "href" in anchor.attrs:
          link = anchor.attrs["href"]
        else:
          link = ''
        
        
        if link.startswith('/'):
              # print(link)
              link = base_url + link
          
          
        elif not link.startswith('http'):
              
              link = path + link

        len1 = len(link)
        # if(link[len1-1] == '#' ):
        #   break
          
        if not link.endswith(".gz"):
          if not link in unscraped and not link in scraped:
              unscraped.append(link)

        # print("links are")
        #print(link)
        
      
      
      new_emails_com = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.com", response.text, re.I))
      new_emails_org = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.org", response.text, re.I))
      emails.update(new_emails_org)
      emails.update(new_emails_com)
      count = count +1
  
  e = []
  for x in emails:
    e.append(x)
  
  final_emails.append(e)
  
   
print(final_emails)
    




Crawling URL http://www.batyr.com.au
Crawling URL http://www.batyr.com.au#
Crawling URL http://www.batyr.com.au##
Crawling URL http://www.batyr.com.au###
Crawling URL http://www.batyr.com.au####
Crawling URL http://www.batyr.com.au#####
Crawling URL http://www.batyr.com.au######
Crawling URL http://www.batyr.com.au#######
Crawling URL http://www.batyr.com.au########
Crawling URL http://www.batyr.com.au#########
Crawling URL http://www.batyr.com.au##########
Crawling URL http://www.aidafghanistanforeducation.org
Crawling URL http://www.aidafghanistanforeducation.org#
Crawling URL http://www.aidafghanistanforeducation.org/contact-us/
Crawling URL http://www.aidafghanistanforeducation.org##
Crawling URL http://www.aidafghanistanforeducation.org/contact-us/#
Crawling URL http://www.aidafghanistanforeducation.org###
Crawling URL http://www.aidafghanistanforeducation.org####
Crawling URL http://www.aidafghanistanforeducation.org#####
Crawling URL http://www.aidafghanistanforeducation.org####

In [7]:
import csv

with open("new_file.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(final_emails)